I will put various functions to compute statistics on slides and pyramids in here

In [1]:
import pymongo
client = pymongo.MongoClient('localhost',27017)
db_ptr = client['TCGA_Slide_DB']
load_errors_db = client['CDSA_LoadErrors']

ImportError: No module named pymongo

In [2]:
## COUND SLIDES IN THE COLLECTION
collection_list = [ x for x in db_ptr.collection_names() if x!= 'system.indexes' ]
print collection_list


svs_summary_dict = {}

for cl in collection_list:
    total_slides =  db_ptr[cl].count()
    
    svs_summary_dict[cl] = {}
    
    frozen_sects =  db_ptr[cl].count({'prep_type': 'tissue_image'})
    svs_summary_dict[cl]['frozen_sects'] =  frozen_sects
    
    diag_sects =  db_ptr[cl].count({'prep_type': 'diagnostic'})
    svs_summary_dict[cl]['diag_sects'] = diag_sects
    
    unknown_sects =  db_ptr[cl].count({'prep_type': 'Unknown'})
    print "%s total: %d frozen sects: %d diagnostic: %d unknown: %d" % ( cl, total_slides, frozen_sects,diag_sects, unknown_sects)
#db_ptr['prad'].find_one()

[u'prad', u'acc', u'gbm', u'lgg', u'coad', u'luad', u'lusc', u'meso', u'cesc', u'ucec', u'read', u'thca', u'brca', u'kirc', u'kirp', u'sarc', u'paad', u'dlbc', u'pcpg', u'blca', u'thym', u'ucs', u'kich', u'stad', u'chol', u'hnsc', u'ov', u'tgct', u'skcm', u'esca', u'lihc']
prad total: 1278 frozen sects: 770 diagnostic: 508 unknown: 0
acc total: 329 frozen sects: 96 diagnostic: 233 unknown: 0
gbm total: 2503 frozen sects: 1435 diagnostic: 1068 unknown: 0
lgg total: 1776 frozen sects: 866 diagnostic: 910 unknown: 0
coad total: 2014 frozen sects: 1547 diagnostic: 467 unknown: 0
luad total: 2104 frozen sects: 1540 diagnostic: 564 unknown: 0
lusc total: 2057 frozen sects: 1478 diagnostic: 579 unknown: 0
meso total: 183 frozen sects: 88 diagnostic: 95 unknown: 0
cesc total: 724 frozen sects: 442 diagnostic: 282 unknown: 0
ucec total: 2142 frozen sects: 1547 diagnostic: 595 unknown: 0
read total: 906 frozen sects: 744 diagnostic: 162 unknown: 0
thca total: 1384 frozen sects: 834 diagnostic: 5

In [77]:
diag_slides =0
frozen_sects = 0
for k in svs_summary_dict:
    diag_slides += svs_summary_dict[k]['diag_sects']
    frozen_sects += svs_summary_dict[k]['frozen_sects']
print diag_slides, frozen_sects

8868 19855


In [85]:
error_db = client['CDSA_LoadErrors']
print error_db.collection_names()
cs_l =  error_db['corrupt_slides'].distinct('full_file_name')

for c in cs_l:
    print c

[u'system.indexes', u'rescanned_slides', u'corrupt_slides', u'Pyramid_Dups']
/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/acc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_ACC.diagnostic_images.Level_1.304.3.0/.TCGA-OR-A5JL-01Z-00-DX3.89E8DA21-12FB-4263-8452-B64F0ED558D8.svs.9gSgQu
/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/acc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_ACC.diagnostic_images.Level_1.304.3.0/.TCGA-OR-A5JP-01Z-00-DX1.15ED1958-011D-4247-91A3-99893466935D.svs.vSanGz
/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/gbm/bcr/intgen.org/diagnostic_images/slide_images/intgen.org_GBM.diagnostic_images.Level_1.5.1.0/TCGA-02-0321-01Z-00-DX1.3c57c27d-4a4f-45af-bf8b-2bde421a42fa.svs.dzi.tif
/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nati

In [5]:
gbm_path_test = '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/'
import os
import timeit

import time

class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start

        
with Timer() as t:
    my_svs_list = [f for root,dirs,files in os.walk(gbm_path_test) for f in files if f.endswith('.svs')] 
print('Request took %.03f sec.' % t.interval)
        
### PROABABLY SHOULD ADD LEAST ADD THE BCR DIRECTORY SO I DON"T SEARCH ALL OTHER PATHS
## SEARCHING JUST GBM/bcr/intgen  TOOK  3.61 seconds
## Seaching ALL of GBM took 46 seconds
## Now lets add BCR and see how long it takes  32.576 seconds, so definitely quicker

Request took 43.993 sec.


In [ ]:
### NEEED TO ADD A FUNCTION TO INSERT METADATA IF THE KEY ISN"T THERE
## AND ADD A FORCE_UPDATE FLAG if the new tag doesn't match the old data in case of a dictionary
        
with Timer() as t:
    my_svs_list = [f for root,dirs,files in os.walk(gbm_path_test+'bcr/') for f in files if f.endswith('.svs')] 
print('Request took %.03f sec.' % t.interval)

In [ ]:
#db_ptr['prad'].find_one({'slide_name': 'TCGA-G9-7523-01A-01-TS1.946b3311-92c7-4023-aed3-16d4280a9725.svs'})

In [105]:
## GET SOME INFO ON DUPLICATE PYRAMIDS-- PROBABLY CAN JUST DELETE THEM
import os,sys
load_errors_db.collection_names()

dup_cur = load_errors_db['Pyramid_Dups'].find()

f_del_out = open('pyrs_that_are_dupes.txt','w')

dup_pyramids = 0
for dc in dup_cur:
    print dc
    dup_pyramids +=  os.path.getsize(dc['pyr_thats_duped'])
    #print dc
    #f_del_out.write( dc['pyr_thats_duped']+'\n'   ) 
    #sys.exit()
#    sys.exit()
print dup_pyramids

0


In [92]:
test_me = '/PYRAMIDS/PYRAMIDS/CDSA/DLBC_Frozen/intgen.org_DLBC.tissue_images.Level_1.248.0.0/TCGA-FA-8693-01A-01-BS1.263ab76a-8168-4f72-8147-8c2a44d0948b.svs.dzi.tif'
pyr_name = ''



pyramid_db = client['CDSA_Pyramid_DB']
print pyramid_db['DLBC_Frozen'].find_one({'pyramid_w_path': test_me})

{u'slide_md5': None, u'pyramid_name': u'TCGA-FA-8693-01A-01-BS1.263ab76a-8168-4f72-8147-8c2a44d0948b.svs.dzi.tif', u'height': 87735, u'width': 73304, u'pyramid_properties': {u'openslide_level[1]_width': u'36652', u'openslide_level[8]_tile-height': u'256', u'openslide_level[0]_tile-height': u'256', u'tiff_XResolution': u'10', u'openslide_level[6]_tile-height': u'256', u'openslide_level[3]_tile-height': u'256', u'openslide_level[6]_height': u'1370', u'openslide_level[7]_tile-height': u'256', u'openslide_level[7]_downsample': u'128.11706906232456', u'openslide_level[3]_downsample': u'8.0003191683384998', u'openslide_level[7]_height': u'685', u'openslide_level[6]_downsample': u'64.030553342045721', u'openslide_level[3]_width': u'9163', u'openslide_level[6]_tile-width': u'256', u'openslide_level-count': u'10', u'openslide_vendor': u'generic-tiff', u'openslide_level[4]_tile-height': u'256', u'openslide_level[1]_tile-height': u'256', u'openslide_level[0]_width': u'73304', u'openslide_level[4]